In [1]:
import sklearn
import numpy as np
import pandas as pd
import re

In [2]:
#training data
train=pd.read_csv("train.csv")

#test data
test=pd.read_csv("test.csv")

In [3]:
train['text length'] = train['tweet'].apply(len)
print(train.head())

   class                                              tweet  text length
0      1  [9-1-13] 2:50 pm "son of a bitch ate my mac n ...           76
1      1  RT @BryceSerna: Don't be a pussy grab the boot...           86
2      2  RT @ClicquotSuave: bunch of rappers boutta flo...           79
3      2  @michigannews13 wow. Thats great language comi...          129
4      1  and this is why I'm single, I don't fuck with ...           77


In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 2),max_df=0.75, min_df=5, max_features=10000)

# TF-IDF feature matrix
tfidf = tfidf_vectorizer.fit_transform(train['tweet'] )
tfidf

<14869x7659 sparse matrix of type '<class 'numpy.float64'>'
	with 222730 stored elements in Compressed Sparse Row format>

In [5]:
!pip install tweet.preprocessor

In [6]:
# remove special characters using the regular expression library
import re

#set up punctuations we want to be replaced
REPLACE_NO_SPACE = re.compile("(\.)|(\;)|(\:)|(\!)|(\')|(\?)|(\,)|(\")|(\|)|(\()|(\))|(\[)|(\])|(\%)|(\$)|(\>)|(\<)|(\{)|(\})")
REPLACE_WITH_SPACE = re.compile("(<br\s/><br\s/?)|(-)|(/)|(:).")

In [7]:
import preprocessor as p

# custum function to clean the dataset (combining tweet_preprocessor and reguar expression)
def clean_tweets(df):
  tempArr = []
  for line in df:
    # send to tweet_processor
    tmpL = p.clean(line)
    # remove puctuation
    tmpL = REPLACE_NO_SPACE.sub("", tmpL.lower()) # convert all tweets to lower cases
    tmpL = REPLACE_WITH_SPACE.sub(" ", tmpL)
    tempArr.append(tmpL)
  return tempArr

In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
X = tfidf
y = train['class'].astype(int)
X_train_tfidf, X_test_tfidf, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.25)
model = LogisticRegression().fit(X_train_tfidf,y_train)


C:\Users\User\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [9]:
import re
import nltk
from nltk.corpus import stopwords
#nltk.download('stopwords')
df_test=pd.read_csv('test.csv')
# df_test=preprocess(df_test)
#test_tweet=df_test.tweet
processed_test_tweets = clean_tweets(df_test.tweet)   
df_test['processed_test_tweets'] = processed_test_tweets
print(df_test[["processed_test_tweets"]])

                                  processed_test_tweets
0      rt dawg rt  you ever fuck a bitch and she sta...
1                            rt  she look like a tranny
2      i can not just sit up and hate on another bit...
3     & cause im tired of you big bitches coming for...
4                  hobbies include fighting mariambitch
...                                                 ...
9909                                  youre all niggers
9910  yous a muthaf***in lie & right his tl is trash...
9911  young buck wanna eat dat nigguh like i aint fu...
9912              youu got wild bitches tellin you lies
9913  ~~ruffled  ntac eileen dahlia   beautiful colo...

[9914 rows x 1 columns]


In [10]:
from sklearn.ensemble import RandomForestClassifier
X_train_tfidf, X_test_tfidf, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)
rf=RandomForestClassifier()
rf.fit(X_train_tfidf,y_train)

RandomForestClassifier()

In [11]:
df_test=pd.read_csv('test.csv')
# df_test=preprocess(df_test)
#test_tweet=df_test.tweet
processed_test_tweets = clean_tweets(df_test.tweet)   
df_test['processed_test_tweets'] = processed_test_tweets
print(df_test[["processed_test_tweets"]])

                                  processed_test_tweets
0      rt dawg rt  you ever fuck a bitch and she sta...
1                            rt  she look like a tranny
2      i can not just sit up and hate on another bit...
3     & cause im tired of you big bitches coming for...
4                  hobbies include fighting mariambitch
...                                                 ...
9909                                  youre all niggers
9910  yous a muthaf***in lie & right his tl is trash...
9911  young buck wanna eat dat nigguh like i aint fu...
9912              youu got wild bitches tellin you lies
9913  ~~ruffled  ntac eileen dahlia   beautiful colo...

[9914 rows x 1 columns]


In [12]:
# clean the test data and append the cleaned tweets to the test data
test_tweet = clean_tweets(test["tweet"])
test_tweet = pd.DataFrame(test_tweet)
# append cleaned tweets to the training data
test["clean_tweet"] = test_tweet

# compare the cleaned and uncleaned tweets
test.tail()

,id,tweet,clean_tweet
9909,9909,you're all niggers,youre all niggers
9910,9910,you's a muthaf***in lie &#8220;@LifeAsKing: @2...,yous a muthaf***in lie & right his tl is trash...
9911,9911,young buck wanna eat!!.. dat nigguh like I ain...,young buck wanna eat dat nigguh like i aint fu...
9912,9912,youu got wild bitches tellin you lies,youu got wild bitches tellin you lies
9913,9913,~~Ruffled | Ntac Eileen Dahlia - Beautiful col...,~~ruffled ntac eileen dahlia beautiful colo...


In [13]:
from sklearn.ensemble import RandomForestClassifier
X_train_tfidf, X_test_tfidf, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)
rf=RandomForestClassifier()
rf.fit(X_train_tfidf,y_train)

RandomForestClassifier()

In [14]:
y_preds = model.predict(X_test_tfidf)
report = classification_report( y_test, y_preds )
print(report)
acc=accuracy_score(y_test,y_preds)
print("Logistic Regression, Accuracy Score:" , acc)

              precision    recall  f1-score   support

           0       0.71      0.09      0.16       167
           1       0.88      0.98      0.93      2287
           2       0.85      0.70      0.77       520

    accuracy                           0.88      2974
   macro avg       0.82      0.59      0.62      2974
weighted avg       0.87      0.88      0.86      2974

Logistic Regression, Accuracy Score: 0.8772696704774714


In [15]:
y_preds = rf.predict(X_test_tfidf)
acc1=accuracy_score(y_test,y_preds)
report = classification_report( y_test, y_preds )
print(report)
print("Random Forest, Accuracy Score:",acc1)

              precision    recall  f1-score   support

           0       0.50      0.05      0.09       167
           1       0.88      0.98      0.93      2287
           2       0.86      0.70      0.77       520

    accuracy                           0.88      2974
   macro avg       0.75      0.57      0.59      2974
weighted avg       0.86      0.88      0.85      2974

Random Forest, Accuracy Score: 0.8755884330867518


In [16]:
from sklearn.naive_bayes import GaussianNB
X_train_tfidf, X_test_tfidf, y_train, y_test = train_test_split(X.toarray(), y, random_state=42, test_size=0.2)
nb=GaussianNB()
nb.fit(X_train_tfidf,y_train)
y_preds = nb.predict(X_test_tfidf)
acc2=accuracy_score(y_test,y_preds)
report = classification_report( y_test, y_preds )
print(report)
print("Naive Bayes, Accuracy Score:",acc2)

              precision    recall  f1-score   support

           0       0.14      0.25      0.18       167
           1       0.86      0.85      0.85      2287
           2       0.65      0.50      0.56       520

    accuracy                           0.76      2974
   macro avg       0.55      0.53      0.53      2974
weighted avg       0.78      0.76      0.77      2974

Naive Bayes, Accuracy Score: 0.7568930733019502


In [17]:
from sklearn.svm import LinearSVC
support =LinearSVC(random_state=20)
support.fit(X_train_tfidf,y_train)
y_preds = support.predict(X_test_tfidf)
acc3=accuracy_score(y_test,y_preds)
report = classification_report( y_test, y_preds )
print(report)
print("SVM, Accuracy Score:" , acc3)

              precision    recall  f1-score   support

           0       0.44      0.19      0.26       167
           1       0.91      0.96      0.93      2287
           2       0.85      0.81      0.83       520

    accuracy                           0.89      2974
   macro avg       0.73      0.65      0.67      2974
weighted avg       0.87      0.89      0.88      2974

SVM, Accuracy Score: 0.8880295897780767
